<a href="https://colab.research.google.com/github/Phoebe125/CRR_and_MonteCarlo/blob/main/CRR_and_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
#Black Scholes Formula
def BS_formula(option_type, S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
      price = S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    elif option_type == 'put':
      price = - S * stats.norm.cdf(-d1) + K * np.exp(-r * T) * stats.norm.cdf(-d2)
    return price


In [ ]:
#Greeks
def delta(option_type, S, K, r, T, sigma):
  d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
  if option_type == 'call':
    return stats.norm.cdf(d1)
  elif option_type == 'put':
    return stats.norm.cdf(d1) - 1

def theta(option_type, S, K, r, T, sigma):
  d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  if option_type == 'call':
    theta = (-sigma * S * stats.norm.pdf(d1)) / (2 * np.sqrt(T)) - r * K * np.exp(-r*T) * stats.norm.cdf(d2)
  elif option_type == 'put':
    theta = (-sigma * S * stats.norm.pdf(d1)) / (2 * np.sqrt(T)) + r * K * np.exp(-r*T) * stats.norm.cdf(-d2)
  return theta

def gamma(S,K,r,T,sigma):
  d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
  gamma = stats.norm.pdf(d1) / (sigma * S * np.sqrt(T))
  return gamma

def vega(S,K,r,T,sigma):
  d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
  vega = S * np.sqrt(T) * stats.norm.pdf(d1)
  return vega

def rho(option_type, S, K, r, T, sigma):
  d1 = (np.log(S/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  if option_type == 'call':
    rho = K * T * np.exp(-r * T) * stats.norm.cdf(d2)
  elif option_type == 'put':
    rho = - K * T * np.exp(-r * T) * stats.norm.cdf(-d2)
  return rho

CRR Model European option pricing, 이항 트리 모형

In [ ]:
def call_payoff(S,K):
  if S > K:
    return (S-K)
  else:
    return 0

def put_payoff(S,K):
  if S < K:
    return (K-S)
  else:
    return 0

In [ ]:
def CRR_european(option_type, S0, K, r, T, sigma, n):

  S = np.zeros((n+1, n+1))
  V = np.zeros((n+1, n+1))
  dt = 1 / n

  a = np.exp(r * dt)
  u = np.exp(sigma * np.sqrt(dt))
  d = 1 / u
  p = (a - d) / (u - d)

  S[0,0] = S0 #현재 가격
  for i in range(1, n+1):
    for j in range(n, 0, -1):
      S[j, i] = u * S[j-1, i-1] #i시점에 j는 위로 몇번 올라갔는지
    S[0, i] = d * S[0, i-1] #한 번도 안올라감

  if option_type == 'call':
    for i in range(n+1):
      V[i] = call_payoff(S[i,n],K) #맨 마지막 payoff나옴
    
    for i in range(n, 0, -1):
      for j in range(0,n):
        V[j, i-1] = ((p * V[j+1, i]) + (1-p) * V[j, i]) * np.exp(-r * dt) #(i-1)시점의 노드값 구함

  if option_type == 'put':
    for i in range(n+1):
      V[i] = put_payoff(S[i,n],K) #맨 마지막 payoff나옴
    
    for i in range(n, 0, -1):
      for j in range(0,n):
        V[j, i-1] = ((p * V[j+1, i]) + (1-p) * V[j, i]) * np.exp(-r * dt) #(i-1)시점의 노드값 구함

  P = pd.DataFrame(S)
  F = pd.DataFrame(V)

  
  return V[0,0], P, F #V[0,0]: 구하고 싶은 현재 옵션의 가격

In [ ]:
S = 100
K = 100
r = 0.05
T = 1
sigma = 0.3
n = 253

In [ ]:
V, P, F = CRR_european('call', S, K, r, T, sigma, n)
print('CRR price: ',V)
print('BS price: ', BS_formula('call', S, K, r, T, sigma))

#점점 n의 수를 늘릴수록 Black Scholes에 수렴해 간다.

CRR price:  14.242110925239784
BS price:  14.231254785985819


In [ ]:
P

,0,1,2,3,4,5,6,7,8,9,...,244,245,246,247,248,249,250,251,252,253
0,100.0,98.131591,96.298092,94.498851,92.733226,91.000590,89.300328,87.631833,85.994512,84.387783,...,1.003131,0.984388,0.965996,0.947947,0.930235,0.912855,0.895799,0.879062,0.862637,0.846520
1,0.0,101.903983,100.000000,98.131591,96.298092,94.498851,92.733226,91.000590,89.300328,87.631833,...,1.041693,1.022230,1.003131,0.984388,0.965996,0.947947,0.930235,0.912855,0.895799,0.879062
2,0.0,0.000000,103.844217,101.903983,100.000000,98.131591,96.298092,94.498851,92.733226,91.000590,...,1.081738,1.061527,1.041693,1.022230,1.003131,0.984388,0.965996,0.947947,0.930235,0.912855
3,0.0,0.000000,0.000000,105.821393,103.844217,101.903983,100.000000,98.131591,96.298092,94.498851,...,1.123322,1.102334,1.081738,1.061527,1.041693,1.022230,1.003131,0.984388,0.965996,0.947947
4,0.0,0.000000,0.000000,0.000000,107.836214,105.821393,103.844217,101.903983,100.000000,98.131591,...,1.166505,1.144710,1.123322,1.102334,1.081738,1.061527,1.041693,1.022230,1.003131,0.984388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10954.646121,10749.968573,10549.115239,10352.014665,10158.596735
250,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11163.220694,10954.646121,10749.968573,10549.115239
251,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11375.766491,11163.220694,10954.646121
252,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11592.359123,11375.766491


In [ ]:
F

,0,1,2,3,4,5,6,7,8,9,...,244,245,246,247,248,249,250,251,252,253
0,14.242111,13.066478,11.957774,10.914812,9.936247,9.020577,8.166150,7.371169,6.633701,5.951685,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,16.732802,15.420905,14.178019,13.003273,11.895638,10.853922,9.876776,8.962690,8.110004,7.316907,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,19.500314,18.048559,16.667277,15.355903,14.113723,12.939869,11.833308,10.792847,9.817130,8.904641,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,22.550720,20.956728,19.434075,17.982485,16.601543,15.290692,14.049221,12.876262,11.770781,10.731584,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,25.887765,24.150278,22.484465,20.890299,19.367635,17.916204,16.535600,15.225272,13.984511,12.812450,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,11694.592763,...,11539.729663,11462.860192,11357.295842,11219.225315,11049.212892,10854.725141,10650.027844,10449.154756,10252.034426,10058.596735
250,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,11699.210747,...,11651.794687,11620.994935,11572.729878,11499.561923,11393.311859,11247.656356,11063.279965,10854.685639,10649.988334,10449.115239
251,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,11703.830554,...,11694.653767,11687.131002,11673.747806,11650.314420,11610.130380,11543.161250,11436.088004,11275.806009,11063.240455,10854.646121
252,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,11708.452185,...,11707.616525,11706.778780,11705.101200,11701.741854,11695.014781,11681.543851,11654.568381,11600.550140,11492.378884,11275.766491


CRR Greeks

In [ ]:
def CRR_delta(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_european(option_type, S, K, r, T, sigma, n)

  fu = F[1][1] #up했을때, pandas: 열-행 순
  fd = F[1][0] #down했을 때
  Su = P[1][1]
  Sd = P[1][0]

  delta = (fu - fd) /(Su - Sd)
  return delta

In [ ]:
print('CRR delta: ', CRR_delta('call', S, K, r, T, sigma, n))
print('BS delta: ', delta('call', S, K, r, T, sigma))

CRR delta:  0.6241205211273427
BS delta:  0.6242517279060125


In [ ]:
def CRR_gamma(S, K, r, T, sigma, n):
  V, P, F = CRR_european('call', S, K, r, T, sigma, n) #감마같은 경우, 콜, 풋 모두 동일해서

  delta_u = (F[2][2] - F[2][1]) / (P[2][2] - P[0][0])
  delta_d = (F[2][1] - F[2][0]) / (P[0][0] - P[2][0])
  h = 0.5 * (P[2][2] - P[2][0])

  gamma = (delta_u - delta_d) / h
  return gamma

In [ ]:
print('CRR gamma: ', CRR_gamma(S, K, r, T, sigma, n))
print('BS gamma: ', gamma( S, K, r, T, sigma))

CRR gamma:  0.012662414047839413
BS gamma:  0.012647764437231512


과제 1: CRR 모형을 이용하여 theta, vega, rho를 구하세요

In [ ]:
def CRR_theta(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_european(option_type, S, K, r, T, sigma, n)

  F2 = F[2][1] #f2,1의 값
  F0 = F[0][0] #f0,0의 값
  delta_t = T / n #델타t의 값

  theta = (F2 - F0) / (2 * delta_t)
  return theta

In [ ]:
print('CRR theta: ', CRR_theta('call', S, K, r, T, sigma, n))
print('BS theta: ', theta('call', S, K, r, T, sigma))

CRR theta:  -8.107616177869868
BS theta:  -8.101189896984952


In [ ]:
import random
def CRR_vega(S, K, r, T, sigma, n):
  V, P, F = CRR_european('call', S, K, r, T, sigma, n) #베가는 콜, 풋 상관 없음
  new_sigma = random.randint(10,99) / 100 #새로운 시그마: 0.01~0.99사이의 임의의 실수
  delta_sigma = new_sigma - sigma  #시그마 차이

  #새로운 시그마로 만든 트리
  new_V, new_P, new_F = CRR_european('call', S, K, r, T, new_sigma, n) #여기서 time step 수는 같아야한다.

  vega = (new_V - V) / delta_sigma
  return vega 

In [ ]:
print('CRR vega: ', CRR_vega(S, K, r, T, sigma, n))
print('BS vega: ', vega(S, K, r, T, sigma))

CRR vega:  37.97273764313623
BS vega:  37.94329331169454


In [ ]:
def CRR_rho(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_european(option_type, S, K, r, T, sigma, n) #베가는 콜, 풋 상관 없음
  new_r = random.randint(1,10) / 100 #새로운 r: 0.01~0.1사이의 임의의 실수
  delta_r = new_r - r

  #새로운 r로 만든 트리
  new_V, new_P, new_F = CRR_european(option_type , S, K, new_r, T, sigma, n) 

  rho = (new_V - V)/delta_r
  return rho 

In [ ]:
print('CRR rho: ', CRR_rho('put', S, K, r, T, sigma, n))
print('BS rho: ', rho('put', S, K, r, T, sigma))

CRR rho:  -49.59382794693691
BS rho:  -46.92902444545599


과제 2: 위의 코드를 참조해서 CRR모형을 이용하여 American option의 가격을 구해보세요

In [ ]:
def CRR_american(option_type, S0, K, r, T, sigma, n):

  S = np.zeros((n+1, n+1))
  V = np.zeros((n+1, n+1))
  dt = 1 / n

  a = np.exp(r * dt)
  u = np.exp(sigma * np.sqrt(dt))
  d = 1 / u
  p = (a - d) / (u - d)

  S[0,0] = S0 #현재 가격
  for i in range(1, n+1):
    for j in range(n, 0, -1):
      S[j, i] = u * S[j-1, i-1] #i시점에 j는 위로 몇번 올라갔는지
    S[0, i] = d * S[0, i-1] #한 번도 안올라감

  if option_type == 'call':
    for i in range(n+1):
      V[i] = call_payoff(S[i,n],K) #맨 마지막 payoff나옴
    
    for i in range(n, 0, -1):
      for j in range(0,n):
        #American이라 early exercise가 가능
        V[j, i-1] = max(((p * V[j+1, i]) + (1-p) * V[j, i]) * np.exp(-r * dt), call_payoff(S[j, i-1],K)) #(i-1)시점의 노드값 구함

  if option_type == 'put':
    for i in range(n+1):
      V[i] = put_payoff(S[i,n],K) #맨 마지막 payoff나옴
    
    for i in range(n, 0, -1):
      for j in range(0,n):
        #American이라 early exercise가 가능
        V[j, i-1] = max(((p * V[j+1, i]) + (1-p) * V[j, i]) * np.exp(-r * dt), put_payoff(S[j, i-1],K)) #(i-1)시점의 노드값 구함

  P = pd.DataFrame(S)
  F = pd.DataFrame(V)

  
  return V[0,0], P, F #V[0,0]: 구하고 싶은 현재 옵션의 가격

In [ ]:
S = 100
K = 100
r = 0.05
T = 1
sigma = 0.3
n = 253

In [ ]:
V_e, P_e, F_e = CRR_european('put', S, K, r, T, sigma, n)
V_a, P_a, F_a = CRR_american('put', S, K, r, T, sigma, n)
print('CRR European price: ',V_e)
print('CRR American price: ', V_a)
print('BS price: ', BS_formula('put', S, K, r, T, sigma))

#점점 n의 수를 늘릴수록 Black Scholes에 수렴해 간다.

CRR European price:  9.36505337531095
CRR American price:  9.880270051983489
BS price:  9.354197236057232


In [ ]:
F_e

,0,1,2,3,4,5,6,7,8,9,...,244,245,246,247,248,249,250,251,252,253
0,9.365053,10.076630,10.820230,11.595318,12.401189,13.236970,14.101626,14.993963,15.912642,16.856186,...,98.819162,98.857634,98.895760,98.933546,98.970999,99.008125,99.044930,99.081420,99.117602,99.153480
1,8.011528,8.658665,9.338567,10.051038,10.795714,11.572056,12.379353,13.216726,14.083129,14.977359,...,98.780599,98.819792,98.858625,98.897105,98.935239,98.973033,99.010494,99.047627,99.084440,99.120938
2,6.787042,7.368910,7.983607,8.631276,9.311892,10.025261,10.771019,11.548623,12.357357,13.196335,...,98.740555,98.780495,98.820063,98.859263,98.898104,98.936592,98.974733,99.012535,99.050004,99.087145
3,5.691990,6.209074,6.758409,7.340448,7.955495,8.603693,9.285025,9.999297,10.746142,11.525017,...,98.698970,98.739688,98.780018,98.819967,98.859542,98.898750,98.937599,98.976094,99.014244,99.052053
4,4.724217,5.178237,5.663341,6.180258,6.729589,7.311795,7.927187,8.575916,9.257964,9.973143,...,98.655787,98.697312,98.738433,98.779159,98.819497,98.859453,98.899036,98.938252,98.977109,99.015612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
F_a

,0,1,2,3,4,5,6,7,8,9,...,244,245,246,247,248,249,250,251,252,253
0,9.88027,10.648374,11.453648,12.295906,13.174808,14.089853,15.040393,16.025633,17.044644,18.096368,...,98.996869,99.015612,99.034004,99.052053,99.069765,99.087145,99.104201,99.120938,99.137363,99.153480
1,100.00000,9.117677,9.848992,10.617676,11.423702,12.266882,13.146868,14.063155,15.015086,16.001854,...,98.958307,98.977770,98.996869,99.015612,99.034004,99.052053,99.069765,99.087145,99.104201,99.120938
2,100.00000,100.000000,8.391494,9.085807,9.817532,10.586800,11.393583,12.237691,13.118772,14.036314,...,98.918262,98.938473,98.958307,98.977770,98.996869,99.015612,99.034004,99.052053,99.069765,99.087145
3,100.00000,100.000000,100.000000,7.701949,8.359203,9.053752,9.785887,10.555742,11.363289,12.208333,...,98.876678,98.897666,98.918262,98.938473,98.958307,98.977770,98.996869,99.015612,99.034004,99.052053
4,100.00000,100.000000,100.000000,100.000000,7.049114,7.669410,8.326725,9.021509,9.754056,10.524501,...,98.833495,98.855290,98.876678,98.897666,98.918262,98.938473,98.958307,98.977770,98.996869,99.015612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000
250,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000
251,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,0.000000,0.000000
252,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,0.000000


과제 3: CRR모형을 사용하여 American option의 Greeks를 구해보세요

In [ ]:
def CRR_delta_american(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_american(option_type, S, K, r, T, sigma, n)

  fu = F[1][1] #up했을때, pandas: 열-행 순
  fd = F[1][0] #down했을 때
  Su = P[1][1]
  Sd = P[1][0]

  delta = (fu - fd) /(Su - Sd)
  return delta

In [ ]:
print('CRR European delta: ', CRR_delta('put', S, K, r, T, sigma, n))
print('CRR American delta: ', CRR_delta_american('put', S, K, r, T, sigma, n))
print('BS delta: ', delta('put', S, K, r, T, sigma))

CRR European delta:  -0.37587947887265866
CRR American delta:  -0.40576307740447437
BS delta:  -0.37574827209398753


In [ ]:
def CRR_gamma_american(S, K, r, T, sigma, n):
  V, P, F = CRR_american('call', S, K, r, T, sigma, n) #감마같은 경우, 콜, 풋 모두 동일해서

  delta_u = (F[2][2] - F[2][1]) / (P[2][2] - P[0][0])
  delta_d = (F[2][1] - F[2][0]) / (P[0][0] - P[2][0])
  h = 0.5 * (P[2][2] - P[2][0])

  gamma = (delta_u - delta_d) / h
  return gamma

In [ ]:
print('CRR European gamma: ', CRR_gamma(S, K, r, T, sigma, n))
print('CRR American gamma: ', CRR_gamma_american(S, K, r, T, sigma, n))
print('BS gamma: ', gamma(S, K, r, T, sigma))

CRR European gamma:  0.012662414047839413
CRR American gamma:  0.012662414047839413
BS gamma:  0.012647764437231512


In [ ]:
def CRR_theta_american(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_american(option_type, S, K, r, T, sigma, n)

  F2 = F[2][1] #f2,1의 값
  F0 = F[0][0] #f0,0의 값
  delta_t = T / n #델타t의 값

  theta = (F2 - F0) / (2 * delta_t)
  return theta

In [ ]:
print('CRR European theta: ', CRR_theta('put', S, K, r, T, sigma, n))
print('CRR American theta: ', CRR_theta_american('put', S, K, r, T, sigma, n))
print('BS theta: ', theta('put', S, K, r, T, sigma))

CRR European theta:  -3.350528981489162
CRR American theta:  -3.956691440755509
BS theta:  -3.3450427744813815


In [ ]:
import random
def CRR_vega_american(S, K, r, T, sigma, n):
  V, P, F = CRR_american('call', S, K, r, T, sigma, n) #베가는 콜, 풋 상관 없음
  new_sigma = random.randint(10,99) / 100 #새로운 시그마: 0.01~0.99사이의 임의의 실수
  delta_sigma = new_sigma - sigma  #시그마 차이

  #새로운 시그마로 만든 트리
  new_V, new_P, new_F = CRR_european('call', S, K, r, T, new_sigma, n) #여기서 time step 수는 같아야한다.

  vega = (new_V - V) / delta_sigma
  return vega 

In [ ]:
print('CRR European vega: ', CRR_vega(S, K, r, T, sigma, n))
print('CRR American vega: ', CRR_vega_american(S, K, r, T, sigma, n))
print('BS vega: ', vega(S, K, r, T, sigma))

CRR European vega:  37.96427872019477
CRR American vega:  37.67423239374213
BS vega:  37.94329331169454


In [ ]:
def CRR_rho_american(option_type, S, K, r, T, sigma, n):
  V, P, F = CRR_american(option_type, S, K, r, T, sigma, n) #베가는 콜, 풋 상관 없음
  new_r = random.randint(1,10) / 100 #새로운 r: 0.01~0.1사이의 임의의 실수
  delta_r = new_r - r

  #새로운 r로 만든 트리
  new_V, new_P, new_F = CRR_european(option_type , S, K, new_r, T, sigma, n) 

  rho = (new_V - V)/delta_r
  return rho 

In [ ]:
print('CRR European rho: ', CRR_rho('put', S, K, r, T, sigma, n))
print('CRR American rho: ', CRR_rho_american('put', S, K, r, T, sigma, n))
print('BS rho: ', rho('put', S, K, r, T, sigma))

CRR European rho:  -48.703780916793605
CRR American rho:  -37.61288706868169
BS rho:  -46.92902444545599


Monte Carlo simulation

In [ ]:
def MonteCarlo(option_type, S0, K, r, T, sigma, I): #I는 시뮬레이션 횟수
  N = 252
  dt = T / N

  S = np.zeros(((N * T + 1), I))
  S[0] = S0

  for t in range(1, N+1): #1개의 path 생성
    A = np.random.standard_normal(I) #I개의 난수 생성
    S[t] = S[t-1] * np.exp((r -0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * A)

  payoff = np.zeros(I) #path별로 payoff를 넣음
  if option_type == 'call':
    payoff = np.exp(-r * T) * (np.maximum(S[N] - np.array(K), 0))
  
  elif option_type == 'put':
    payoff = np.exp(-r * T) * (np.maximum(np.array(K) - S[N]), 0)

  #plt.plot(S[: 50])

  V = np.mean(payoff)
  return V


In [ ]:
I = 100000
print('monte carlo: ', MonteCarlo('call', S, K, r, T, sigma, I))
print('BS price: ', BS_formula('call', S, K, r, T, sigma))

monte carlo:  14.259476086931608
BS price:  14.231254785985819


과제 4: Monte Carlo Simulation을 이용하여 European option Greeks를 구하세요

In [ ]:
def MC_delta(option_type, S, K, r, T, sigma, I):
  F = MonteCarlo(option_type, S, K, r, T, sigma, I)
  new_S = S * 1.01
  dS = new_S - S
  new_F = MonteCarlo(option_type, new_S, K, r, T, sigma, I)

  return (new_F - F) / dS

In [ ]:
print('monte carlo delta: ', MC_delta('call', S, K, r, T, sigma, I))
print('BS delta: ', delta('call', S, K, r, T, sigma))

monte carlo delta:  0.6439313820819574
BS delta:  0.6242517279060125


In [ ]:
def MC_gamma(S, K, r, T, sigma, I):
  #F = MonteCarlo('call', S, K, r, T, sigma, I)
  delta_F = MC_delta('call', S, K, r, T, sigma, I)

  new_S = S * 1.01
  dS = new_S - S
  #new_F =  MonteCarlo('call', new_S, K, r, T, sigma, I)
  delta_new_F = MC_delta('call', new_S, K, r, T, sigma, I)

  return (delta_new_F - delta_F) / dS

In [ ]:
print('monte carlo gamma: ', MC_gamma(S, K, r, T, sigma, I))
print('BS gamma: ', gamma(S, K, r, T, sigma))

original delta:  0.6459542285795337
new delta:  0.6347198377587507
monte carlo gamma:  -0.011234390820782925
BS gamma:  0.012647764437231512


In [ ]:
def MC_vega(S, K, r, T, sigma, I):
  F = MonteCarlo('call', S, K, r, T, sigma, I)

  new_sigma = random.randint(10,99) / 100 #새로운 시그마: 0.01~0.99사이의 임의의 실수
  delta_sigma = new_sigma - sigma  #시그마 차이
  new_F = MonteCarlo('call', S, K, r, T, new_sigma, I)

  return (new_F - F) / delta_sigma

In [ ]:
print('monte carlo vega: ', MC_vega(S, K, r, T, sigma, I))
print('BS vega: ', vega(S, K, r, T, sigma))

monte carlo vega:  38.0501253655782
BS vega:  37.94329331169454


In [ ]:
def MC_rho(option_type, S, K, r, T, sigma, I):
  F = MonteCarlo('call', S, K, r, T, sigma, I)

  new_r = random.randint(1,9) / 100 #새로운 r: 0.01~0.09사이의 임의의 실수
  delta_r = new_r - r
  new_F = MonteCarlo('call', S, K, new_r, T, sigma, I)

  return (new_F - F) / delta_r

In [ ]:
print('monte carlo rho: ', MC_rho('call', S, K, r, T, sigma, I))
print('BS rho: ', rho('call',S, K, r, T, sigma))

monte carlo rho:  43.49158941929283
BS rho:  48.193918004615426
